In [68]:
"""
network.py
~~~~~~~~~~

A module to implement the stochastic gradient descent learning
algorithm for a feedforward neural network.  Gradients are calculated
using backpropagation.  Note that I have focused on making the code
simple, easily readable, and easily modifiable.  It is not optimized,
and omits many desirable features.
"""

#### Libraries
# Standard library
import random

# Third-party libraries
import numpy as np

class Network(object):

    def __init__(self, sizes):
        """The list ``sizes`` contains the number of neurons in the
        respective layers of the network.  For example, if the list
        was [2, 3, 1] then it would be a three-layer network, with the
        first layer containing 2 neurons, the second layer 3 neurons,
        and the third layer 1 neuron.  The biases and weights for the
        network are initialized randomly, using a Gaussian
        distribution with mean 0, and variance 1.  Note that the first
        layer is assumed to be an input layer, and by convention we
        won't set any biases for those neurons, since biases are only
        ever used in computing the outputs from later layers."""
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(sizes[:-1], sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            test_data=None, validation_data=None):
        """Train the neural network using mini-batch stochastic
        gradient descent.  The ``training_data`` is a list of tuples
        ``(x, y)`` representing the training inputs and the desired
        outputs.  The other non-optional parameters are
        self-explanatory.  If ``test_data`` is provided then the
        network will be evaluated against the test data after each
        epoch, and partial progress printed out.  This is useful for
        tracking progress, but slows things down substantially."""
        if test_data: n_test = len(test_data)
        n = len(training_data)
        for j in xrange(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in xrange(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(mini_batch, eta)
            if test_data:
                print "Epoch {0}: {1} / {2}".format(
                    j, self.evaluate(test_data), n_test)
            else:
                print "Epoch {0} complete".format(j)
                
        if validation_data:
            self.evaluateValidation(validation_data)

    def update_mini_batch(self, mini_batch, eta):
        """Update the network's weights and biases by applying
        gradient descent using backpropagation to a single mini batch.
        The ``mini_batch`` is a list of tuples ``(x, y)``, and ``eta``
        is the learning rate."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = self.cost_derivative(activations[-1], y) * \
            sigmoid_prime(zs[-1])
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # Note that the variable l in the loop below is used a little
        # differently to the notation in Chapter 2 of the book.  Here,
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.  It's a renumbering of the
        # scheme in the book, used here to take advantage of the fact
        # that Python can use negative indices in lists.
        for l in xrange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def evaluate(self, test_data):
        """Return the number of test inputs for which the neural
        network outputs the correct result. Note that the neural
        network's output is assumed to be the index of whichever
        neuron in the final layer has the highest activation."""
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        #print(test_results)
        return sum(int(x == y) for (x, y) in test_results)
    
    def evaluateValidation(self, test_data):
        test_results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in test_data]
        print(test_results)
        #return sum(int(x == y) for (x, y) in test_results)

    def cost_derivative(self, output_activations, y):
        """Return the vector of partial derivatives \partial C_x /
        \partial a for the output activations."""
        return (output_activations-y)

#### Miscellaneous functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

In [69]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from random import randint, randrange
import os
import pickle

def save_images_as_pkl(directory='test_images', file_name='coconut_data.pkl'):
    data = [[], []]
    
    d1 = directory+'/coconut'
    d2 = directory+'/non_coconut'
    for filename in os.listdir(d1):

        img = cv2.imread('%s/%s' % (d1, filename),0)
        #edges = cv2.Canny(img,t1,t2)
        edges = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C,\
                                      cv2.THRESH_BINARY,11,2)
        
        data[0].append(np.hstack(edges))
        data[1].append(0)
    
    for filename in os.listdir(d2):

        img = cv2.imread('%s/%s' % (d2, filename),0)
        #edges = cv2.Canny(img,t1,t2)
        edges = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C,\
                                      cv2.THRESH_BINARY,11,2)
        
        data[0].append(np.hstack(edges))
        data[1].append(1)
    
    output = open(file_name, 'wb')
    pickle.dump(data, output)
    output.close()
    
def load_data_pkl():
    images = []
    cat = []
    
    d_file = open('pickle_data/coconut_data.pkl')
    t_file = open('pickle_test/coconut_data.pkl')
    v_file = open('pickle_validation/coconut_data.pkl')
    d_data = pickle.load(d_file)
    t_data = pickle.load(t_file)
    v_data = pickle.load(v_file)
        
    return (d_data, v_data, t_data)

def load_data_wrapper_pkl():

    tr_d, va_d, te_d = load_data_pkl()
    training_inputs = [np.reshape(x, (12100, 1)) for x in tr_d[0]]
    training_results = [vectorized_result_pkl(y) for y in tr_d[1]]
    training_data = zip(training_inputs, training_results)
    
    validation_inputs = [np.reshape(x, (12100, 1)) for x in va_d[0]]
    validation_data = zip(validation_inputs, va_d[1])
    test_inputs = [np.reshape(x, (12100, 1)) for x in te_d[0]]
    test_data = zip(test_inputs, te_d[1])
    return (training_data, validation_data, test_data)

def vectorized_result_pkl(j):
    e = np.zeros((2, 1))
    e[j] = 1.0
    return e

In [70]:
# produce pickle data from images, saves into coconut_data.pkl
#save_images_as_pkl('images')
#save_images_as_pkl()

In [71]:
training_data, validation_data, test_data = load_data_wrapper_pkl()
net = Network([12100, 30, 2])
net.SGD(training_data, 30, 10, 2.0, test_data=test_data, validation_data=validation_data)

Epoch 0: 20 / 30
Epoch 1: 20 / 30
Epoch 2: 20 / 30
Epoch 3: 20 / 30
Epoch 4: 20 / 30
Epoch 5: 20 / 30
Epoch 6: 20 / 30
Epoch 7: 20 / 30
Epoch 8: 20 / 30
Epoch 9: 20 / 30
Epoch 10: 20 / 30
Epoch 11: 20 / 30
Epoch 12: 20 / 30
Epoch 13: 20 / 30
Epoch 14: 20 / 30
Epoch 15: 20 / 30
Epoch 16: 20 / 30
Epoch 17: 20 / 30
Epoch 18: 20 / 30
Epoch 19: 20 / 30
Epoch 20: 20 / 30
Epoch 21: 20 / 30
Epoch 22: 20 / 30
Epoch 23: 20 / 30
Epoch 24: 20 / 30
Epoch 25: 20 / 30
Epoch 26: 20 / 30
Epoch 27: 20 / 30
Epoch 28: 18 / 30
Epoch 29: 20 / 30
[(0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1)]


/home/rashi/Envs/visualization/lib/python2.7/site-packages/ipykernel/__main__.py:146: RuntimeWarning: overflow encountered in exp


In [53]:
s = [1,2]
print(s)

[1, 2]


In [75]:
net.evaluate(test_data)


/home/rashi/Envs/visualization/lib/python2.7/site-packages/ipykernel/__main__.py:146: RuntimeWarning: overflow encountered in exp


20